In [ ]:
## Code for this part referred to the quantization notebook provided by TA Konan

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import torchvision   
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm
import time
import torch.optim as optim
from torch.autograd import Variable
import os
import zipfile
from torch.utils.data.sampler import SubsetRandomSampler, SequentialSampler

In [ ]:
!pip3 install torch
!pip3 install torchvision
!pip3 install gdown

In [ ]:
# data download source is not shown due to fair use

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(use_cuda, device, torch.cuda.device_count())

## Model

In [ ]:
class MyBetaVAE(nn.Module):
    def __init__(self, in_channels, z_dim, beta): # image should have size 64*64
        super(MyBetaVAE, self).__init__()
        self.z_dim = z_dim
        self.beta = beta

        # hidden_dims = [in_channels, 32, 64, 128, 256, 512]
        hidden_dims = [in_channels, 32, 64, 128, 256]
        self.encoder_final_size = 16
        self.hidden_dims = hidden_dims

        # encoder
        encoder_layers = []
        for i in range(len(hidden_dims)-1):
            encoder_layers.append(nn.Sequential(
                                        nn.Conv2d(hidden_dims[i], hidden_dims[i+1], kernel_size=3, stride=2, padding=1),
                                        nn.BatchNorm2d(hidden_dims[i+1]),
                                        nn.LeakyReLU()
                                  ))
        
        self.encoder = nn.Sequential(*encoder_layers)
        self.mu = nn.Linear(hidden_dims[-1] * self.encoder_final_size, z_dim)
        self.logvar = nn.Linear(hidden_dims[-1] * self.encoder_final_size, z_dim)

        # decoder
        self.decoder_in = nn.Linear(z_dim, hidden_dims[-1] * self.encoder_final_size)

        decoder_layers = []
        for i in range(len(hidden_dims)-1, 0, -1):
            decoder_layers.append(nn.Sequential(
                                      nn.ConvTranspose2d(hidden_dims[i], hidden_dims[i] if i == 1 else hidden_dims[i-1], kernel_size=3, stride=2, padding=1, output_padding=1),
                                      nn.BatchNorm2d(hidden_dims[i] if i == 1 else hidden_dims[i-1]),
                                      nn.LeakyReLU()
                                  ))
        
        self.decoder = nn.Sequential(*decoder_layers,
                                     nn.Conv2d(hidden_dims[1], hidden_dims[0], kernel_size=3, padding=1),
                                     nn.Sigmoid())
    
    def encode(self, x):
        out = self.encoder(x)
        out = torch.flatten(out, start_dim=1)
        mu = self.mu(out)
        logvar = self.logvar(out)
        self.mu_value = mu
        self.logvar_value = logvar
        self.x = x
        return mu, logvar
    
    def decode(self, x):
        out = self.decoder_in(x).view(-1, self.hidden_dims[-1], 4, 4)
        out = self.decoder(out)
        return out
    
    def reparam(self, mu, logvar):
        std = torch.exp(logvar / 2)
        epsilon = torch.autograd.Variable(torch.randn_like(std))
        return std * epsilon + mu


    def forward(self, x):
        time1 = time.time()
        mu, logvar = self.encode(x)
        z = self.reparam(mu, logvar)
        time2 = time.time()
        out = self.decode(z)
        time3 = time.time()
        self.x_cons = out
        return out, z, mu, logvar, Variable(torch.tensor(time2-time1)), Variable(torch.tensor(time3-time2)) #{"encodertime": , "decodertime": }
    
    def loss(self):
        reconstruction_loss = F.mse_loss(self.x_cons, self.x, reduction='sum')
        kl_div = torch.mean(torch.sum(-0.5 * (1 + self.logvar_value - self.mu_value ** 2 - self.logvar_value.exp()), dim=1), dim=0)
        return reconstruction_loss + self.beta * kl_div, reconstruction_loss, kl_div

In [ ]:
class MyBetaVAEEncoder(MyBetaVAE):
    
    def __init__(self, in_channels, z_dim, beta): # image should have size 64*64
        super().__init__(in_channels, z_dim, beta)
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparam(mu, logvar)
        return z, mu, logvar

class MyBetaVAEDecoder(MyBetaVAE):
    
    def __init__(self, in_channels, z_dim, beta): # image should have size 64*64
        super().__init__(in_channels, z_dim, beta)
    
    def forward(self, z):
        out = self.decode(z)
        return out

## DataLoader

In [ ]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.Resize((64, 64)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((64, 64)),
    torchvision.transforms.ToTensor()                                              
])

def get_dataloaders(path, shuffle, portion, val_split, batch_size, test=False):
    dataset = torchvision.datasets.ImageFolder(root=path, transform=transforms)
    if test:
      test_dataset = torchvision.datasets.ImageFolder(root=path, transform=test_transforms)
      for filename in test_dataset.imgs:
          frame_names.append(filename[0].split('/')[-1])

    shuffle_dataset = shuffle
    use_proportion = portion
    validation_split = val_split
    batch_size = batch_size
    random_seed= 42

    # Creating data indices for training and validation splits:
    indices = list(range(len(dataset)))
    use_length = int(len(dataset) * use_proportion)
    split = int(np.floor((1 - validation_split) * len(dataset)) * use_proportion)
    if shuffle_dataset:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices = indices[:split]
    print(train_indices)
    val_indices = indices[split:use_length]
    print(len(train_indices), len(val_indices))
    print(len(dataset), use_length, split)

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)
  

    train_dataloader = DataLoader(dataset, batch_size=batch_size, drop_last=True, sampler=train_sampler, num_workers=4)
    val_dataloader = DataLoader(dataset, batch_size=batch_size, drop_last=True, sampler=val_sampler, num_workers=4)

    if test:
      test_sampler = SequentialSampler(train_indices)
      test_dataloader = DataLoader(test_dataset, batch_size=batch_size, drop_last=False, sampler=test_sampler, num_workers=4)
      return test_dataloader, test_dataloader
    

    return train_dataloader, val_dataloader

## Inference & Accuracy

In [ ]:
!mkdir ./dev_data/

In [ ]:
!unzip ./unique-142p-valid.zip
!mv ./unique-142p-valid ./dev_data/unique-142p-valid

In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt

logs = {}
avg_loss = 0
z_ls = []
batch_size = 512 # 2048, 1024, 512, 256

frame_names = []
dev_loader, dev_original_loader = get_dataloaders('./dev_data', shuffle=False, portion=1, val_split=0, batch_size=batch_size, test=True)
print(frame_names)

In [ ]:
import numpy as np
labels = np.genfromtxt('./scene-change.csv', delimiter=',', names=True, dtype=None, encoding=None)

In [ ]:
len(labels)

In [ ]:
label_ls = [] # record frame1 in true_labels
for labels in labels:
  label_ls.append(labels[0])
print(label_ls[:10])

In [ ]:
# label for scene change 1 represent change
true_labels = []
for name in frame_names:
    if name in label_ls:
        true_labels.append(1)
    else:
        true_labels.append(0)
print(len(true_labels))

## Quantization

In [ ]:
!mkdir models
!pip3 install onnx
!pip3 install onnxruntime
!pip3 install py-cpuinfo
!pip3 install torch-summary

In [ ]:
# Model and Inference
import onnx
import onnx.numpy_helper
import onnxruntime
import onnxruntime.quantization

# Data Loader
import torch
import torchvision

# Evaluation
import numpy as np
import sklearn.metrics
from torchsummary import summary


# Visualization
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

# Other Utilities
import os
import pandas as pd
from tqdm.notebook import tqdm
# import cpuinfo # !conda install -c conda-forge py-cpuinfo # not used in colab
import cpuinfo


print("onnx", onnx.__version__)
print("onnxruntime", onnxruntime.__version__)

In [ ]:
cpuinfo.get_cpu_info()

In [ ]:
!cat /proc/meminfo

In [ ]:
### Get Original PyTorch Model
checkpoint = torch.load("./v2_best_checkpoint_epoch125.pt", map_location=torch.device(device))

in_channels, z_dim, beta = 3, 128, 0.5
model = MyBetaVAE(in_channels, z_dim, beta)
model.load_state_dict(state_dict=checkpoint['model'])
model.to(device)
model_encoder = MyBetaVAEEncoder(in_channels, z_dim, beta)
model_decoder = MyBetaVAEDecoder(in_channels, z_dim, beta)
model_encoder.load_state_dict(state_dict=checkpoint['model'])
model_decoder.load_state_dict(state_dict=checkpoint['model'])
model_encoder.to(device)
model_decoder.to(device)
print(summary(model, (3, 64, 64)))
model.eval()
model_encoder.eval()
model_decoder.eval()

### Save PyTorch Model
PT_MODEL_PATH = "./models/best_model.pt"
torch.save(model, PT_MODEL_PATH)
PT_MODEL_PATH_ENCODER = "./models/best_model_encoder.pt"
torch.save(model, PT_MODEL_PATH_ENCODER)
PT_MODEL_PATH_DECODER = "./models/best_model_decoder.pt"
torch.save(model, PT_MODEL_PATH_DECODER)

In [ ]:
### Convert PyTorch to ONNX Model
dummy_input = torch.ones(batch_size,3,64,64).to(device)
dynamic_axes = {'input' : {0 : 'batch_size'}, 'output' : {0 : 'batch_size'}}
ONNX_MODEL_PATH = "./models/best_model.onnx"
QI_ONNX_MODEL_PATH = "./models/QI-best_model.onnx"
torch.onnx.export(model,
                  dummy_input,
                  ONNX_MODEL_PATH,
                  input_names = ['input'],
                  output_names = ['output'],
                  dynamic_axes=dynamic_axes,
                  opset_version=11)

In [ ]:
dummy_input = torch.ones(batch_size, 3,64,64).to(device)
dynamic_axes = {'input' : {0 : 'batch_size'}, 'output' : {0 : 'batch_size'}}
ONNX_MODEL_PATH_ENCODER = "./models/best_model_encoder.onnx"
QI_ONNX_MODEL_PATH_ENCODER = "./models/QI-best_model_encoder.onnx"
torch.onnx.export(model_encoder,
                  dummy_input,
                  ONNX_MODEL_PATH_ENCODER,
                  input_names = ['input'],
                  output_names = ['output'],
                  dynamic_axes=dynamic_axes,
                  opset_version=11)

In [ ]:
dummy_input = torch.ones(batch_size,128).to(device)
dynamic_axes = {'input' : {0 : 'batch_size'}, 'output' : {0 : 'batch_size'}}
ONNX_MODEL_PATH_DECODER = "./models/best_model_decoder.onnx"
QI_ONNX_MODEL_PATH_DECODER = "./models/QI-best_model_decoder.onnx"
torch.onnx.export(model_decoder,
                  dummy_input,
                  ONNX_MODEL_PATH_DECODER,
                  input_names = ['input'],
                  output_names = ['output'],
                  dynamic_axes=dynamic_axes,
                  opset_version=11)

In [ ]:
kwargs = {"op_types_to_quantize":     [],
          "per_channel":              False,
          "reduce_range":             False,
          "activation_type":          onnxruntime.quantization.QuantType.QUInt8,
          "weight_type":              onnxruntime.quantization.QuantType.QUInt8,
          "nodes_to_quantize":        [],
          "nodes_to_exclude":         [],
          "use_external_data_format": False}

onnxruntime.quantization.quantize_dynamic(ONNX_MODEL_PATH, 
                                          QI_ONNX_MODEL_PATH, 
                                          **kwargs)
onnxruntime.quantization.quantize_dynamic(ONNX_MODEL_PATH_ENCODER, 
                                          QI_ONNX_MODEL_PATH_ENCODER, 
                                          **kwargs)
onnxruntime.quantization.quantize_dynamic(ONNX_MODEL_PATH_DECODER, 
                                          QI_ONNX_MODEL_PATH_DECODER, 
                                          **kwargs)

In [ ]:
onnx_size_mb   = os.path.getsize(ONNX_MODEL_PATH)/1e6
qi_onnx_size_mb = os.path.getsize(QI_ONNX_MODEL_PATH)/1e6

print("\nONNX model size (MB):     {:.2f}".format(onnx_size_mb))
print("\nQI-ONNX model size (MB):  {:.2f}".format(qi_onnx_size_mb))

In [ ]:
class init_inference:
    def __init__(self, session):
        self.session = session
        
    def __call__(self, image):
        input_data = image.numpy() # x
        input_name  = self.session.get_inputs()[0].name
    
        return self.session.run(None, {input_name: input_data})

def total_latency_fn(tqdm_dataloader):
    start = tqdm_dataloader.start_t
    stop = tqdm_dataloader.last_print_t
    n = tqdm_dataloader.n
    
    total_latency = (stop - start)
    
    return total_latency

def simple_table(data_multilist, x_label_list, y_label_list):
    for i in range(len(y_label_list)+1):
        if i == 0:
            row = ["\t"]
            row.extend(x_label_list)
            string = "\t".join(row)
        else:
            row = [y_label_list[i-1]]
            
            row.extend([format(x, ".6f") for x in data_multilist[i-1]])
            string = "\t".join(row)
            
        print(string)
        
    return None

def my_loss(x_cons, x, mu, logvar):
    x_cons = torch.Tensor(x_cons)
    mu = torch.Tensor(mu)
    logvar = torch.Tensor(logvar)
    reconstruction_loss = F.mse_loss(x_cons, x, reduction='sum')
    kl_div = torch.mean(torch.sum(-0.5 * (1 + logvar - mu ** 2 - logvar.exp()), dim=1), dim=0)
    return reconstruction_loss + beta * kl_div, reconstruction_loss, kl_div

def plot_statistic(x, y, title, x_label, y_label):

    label_pos = [i for i, _ in enumerate(x)]

    plt.figure(figsize=(12, 6))
    plt.bar(label_pos, x, color='black', width=0.8)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.xticks(label_pos, y)
    
    plt.show()
    
    return None


def validate(dataloader, session, session_encoder, session_decoder, true_labels):
    tqdm_dataloader = tqdm(dataloader)
    inference = init_inference(session)
    inference_e = init_inference(session_encoder)
    inference_d = init_inference(session_decoder)
    z_ls = []
    cos_ls, l2_ls = [], []
    loss_labels, pred_labels = [], []
    losses, mses, klds = 0, 0, 0
    total_encoder_time = 0
    total_decoder_time = 0
    cos = nn.CosineSimilarity(dim=0, eps=1e-6)

    start_time = time.time()

    for idx, (images, target) in enumerate(tqdm_dataloader):
        
        out, z, mu, logvar, _, _ = inference(images) # out, z, time in secs
        for z_single in z:
            z_ls.append(z_single)
        loss, mse, kld = my_loss(out, images, mu, logvar)

        losses += loss
        mses += mse
        klds += kld
    
    end_time = time.time()

    total_latency = end_time - start_time # secs

    for idx, (images, target) in enumerate(tqdm_dataloader):
        
        start_time = time.time()
        z, mu, logvar = inference_e(images) # z, mu, logvar
        end_time = time.time()
        total_encoder_time += (end_time - start_time)

        start_time = time.time()
        z = torch.tensor(z)
        out = inference_d(z)
        end_time = time.time()
        total_decoder_time += (end_time - start_time)
        print(total_encoder_time, total_decoder_time)
    
    for i in range(len(z_ls) - 1):

        cos_sim = cos(torch.tensor(z_ls[i]), torch.tensor(z_ls[i + 1]))
        cos_ls.append(cos_sim.item())
        l2 = torch.norm(torch.tensor(z_ls[i]) - torch.tensor(z_ls[i + 1]), 2)
        l2_ls.append(l2)
        
        
    # top k prediction
    K = true_labels.sum()
    topk_idx_true = np.argsort(-np.array(true_labels))[:K]
    
    cos_argmin_k_idx = np.argsort(np.array(cos_ls))
    cos_topk_idx = cos_argmin_k_idx[:K]
    
    l2_argmax_k_idx = np.argsort(-np.array(l2_ls))
    l2_topk_idx = l2_argmax_k_idx[:K]

    def predict_topk(topk_idx, topk_idx_true, z_ls):
        pred_labels_topk = [0] * (len(z_ls) - 1)
        for idx in topk_idx:
            pred_labels_topk[idx] = 1
        # compute cos top k accuracy
        correct_num = 0
        for idx in topk_idx:
            if idx in topk_idx_true:
                correct_num += 1
        top_k_acc = correct_num / K

        return pred_labels_topk, top_k_acc


    cos_pred_labels_topk, cos_topk_acc = predict_topk(cos_topk_idx, topk_idx_true, z_ls)
    l2_pred_labels_topk, l2_topk_acc = predict_topk(l2_topk_idx, topk_idx_true, z_ls)


    return losses / len(z_ls), mses / len(z_ls), klds / len(z_ls), total_latency, total_encoder_time, total_decoder_time, cos_pred_labels_topk, cos_topk_acc, l2_pred_labels_topk, l2_topk_acc


In [ ]:
ort_session = onnxruntime.InferenceSession(ONNX_MODEL_PATH)
ort_session_encoder = onnxruntime.InferenceSession(ONNX_MODEL_PATH_ENCODER)
ort_session_decoder = onnxruntime.InferenceSession(ONNX_MODEL_PATH_DECODER)

losses, mses, klds, total_latency, total_encoder_time, total_decoder_time, cos_pred_labels_topk, cos_topk_acc, l2_pred_labels_topk, l2_topk_acc = validate(dev_loader, ort_session, ort_session_encoder, ort_session_decoder, np.array(true_labels))





In [ ]:
losses, mses, klds, total_latency, total_encoder_time, total_decoder_time, cos_topk_acc, l2_topk_acc

In [ ]:
# accuracy_score      = sklearn.metrics.accuracy_score(true_labels[:len(pred_labels)], pred_labels)
# recall_score      = sklearn.metrics.recall_score(true_labels[:len(pred_labels)], pred_labels)
# precision_score      = sklearn.metrics.precision_score(true_labels[:len(pred_labels)], pred_labels)
# print(accuracy_score, recall_score, precision_score)

# sklearn.metrics.classification_report(true_labels[:len(pred_labels)], pred_labels)
print(sklearn.metrics.classification_report(true_labels[:len(cos_pred_labels_topk)], cos_pred_labels_topk))

In [ ]:
print(sklearn.metrics.classification_report(true_labels[:len(l2_pred_labels_topk)], l2_pred_labels_topk))

In [ ]:
qi_ort_session = onnxruntime.InferenceSession(QI_ONNX_MODEL_PATH)
qi_ort_session_encoder = onnxruntime.InferenceSession(QI_ONNX_MODEL_PATH_ENCODER)
qi_ort_session_decoder = onnxruntime.InferenceSession(QI_ONNX_MODEL_PATH_DECODER)

qi_losses, qi_mses, qi_klds, qi_total_latency, qi_total_encoder_time, qi_total_decoder_time, qi_cos_pred_labels_topk, qi_cos_topk_acc, qi_l2_pred_labels_topk, qi_l2_topk_acc = validate(dev_loader, qi_ort_session, qi_ort_session_encoder, qi_ort_session_decoder, np.array(true_labels))




In [ ]:
qi_losses, qi_mses, qi_klds, qi_total_latency, qi_total_encoder_time, qi_total_decoder_time, qi_cos_topk_acc, qi_l2_topk_acc

In [ ]:
# qi_accuracy_score   = sklearn.metrics.accuracy_score(true_labels[:len(qi_pred_labels)], qi_pred_labels)
# qi_recall_score   = sklearn.metrics.recall_score(true_labels[:len(qi_pred_labels)], qi_pred_labels)
# qi_precision_score   = sklearn.metrics.precision_score(true_labels[:len(qi_pred_labels)], qi_pred_labels)
# print(qi_accuracy_score, qi_recall_score, qi_precision_score)

# sklearn.metrics.classification_report(true_labels[:len(qi_pred_labels)], qi_pred_labels)
print(sklearn.metrics.classification_report(true_labels[:len(qi_cos_pred_labels_topk)], qi_cos_pred_labels_topk))

In [ ]:
print(sklearn.metrics.classification_report(true_labels[:len(qi_l2_pred_labels_topk)], qi_l2_pred_labels_topk))

# Not Use

In [ ]:
len(pred_labels)#, len(qi_pred_labels)

In [ ]:
# qi_losses, qi_mses, qi_klds
print('Validation loss results are below.\n')

simple_table(
    data_multilist = [[losses, mses, klds],
                      [qi_losses, qi_mses, qi_klds]],
    x_label_list = ["total loss", "mse        ", "kld"],
    y_label_list = ["Float32 ", "IntegerOps"]
)

In [ ]:
x = [losses, qi_losses, mses, qi_mses, klds, qi_klds]
y = ['Float32-Total Loss','IntegerOps-Total Loss', 'Float32-MSE','IntegerOps-MSE', 'Float32-KLD','IntegerOps-KLD',]
title = "Loss Results"
x_label = "Model Type"
y_label = "Loss"
    
plot_statistic(x, y, title, x_label, y_label)

In [ ]:
# last one in true_labels not used
accuracy_score      = sklearn.metrics.accuracy_score(true_labels[:len(pred_labels)], pred_labels)
qi_accuracy_score   = sklearn.metrics.accuracy_score(true_labels[:len(qi_pred_labels)], qi_pred_labels)

recall_score      = sklearn.metrics.recall_score(true_labels[:len(pred_labels)], pred_labels)
qi_recall_score   = sklearn.metrics.recall_score(true_labels[:len(qi_pred_labels)], qi_pred_labels)

precision_score      = sklearn.metrics.precision_score(true_labels[:len(pred_labels)], pred_labels)
qi_precision_score   = sklearn.metrics.precision_score(true_labels[:len(qi_pred_labels)], qi_pred_labels)

print('Validation accuracy results are below.\n')

simple_table(
    data_multilist = [[accuracy_score, precision_score, recall_score] ,
                      [qi_accuracy_score, qi_precision_score, qi_recall_score]],
    x_label_list = ["Accuracy", "Precision", "Recall"],
    y_label_list = ["Float32 ", "IntegerOps"]
)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(true_labels[:len(pred_labels)], pred_labels), confusion_matrix(true_labels[:len(qi_pred_labels)], qi_pred_labels)

In [ ]:
x = [accuracy_score, qi_accuracy_score, precision_score, qi_precision_score, recall_score, qi_recall_score]
y = ['Accuracy-Float32','Accuracy-IntegerOps', 'Precision-Float32','Precision-IntegerOps', 'Recall-Float32','Recall-IntegerOps']
title = "Scene Change Detection Accuracy, Precision, Recall"
x_label = "Model Type"
y_label = "Metric"
    
plot_statistic(x, y, title, x_label, y_label)

In [ ]:
print('Model sizes are below.\n')

simple_table(
    data_multilist = [[onnx_size_mb] ,
                      [qi_onnx_size_mb]],
    x_label_list = ["Baseline"],
    y_label_list = ["Float32 ", "IntegerOps"]
)

In [ ]:
x = [onnx_size_mb, qi_onnx_size_mb]
y = ['Float32', 'IntegerOps']
title = "Model Size"
x_label = "Model Type"
y_label = "Model Size (MB)"
    
plot_statistic(x, y, title, x_label, y_label)

In [ ]:
print('Validation latency results are below.\n')

simple_table(
    data_multilist = [[total_latency   , total_encoder_time, total_decoder_time],
                      [qi_total_latency, qi_total_encoder_time, qi_total_decoder_time]], 
    x_label_list = ["Total Latency", "Encoder Latency", "Decoder Latency"], 
    y_label_list = ["Float32 ", "IntegerOps"]
)

In [ ]:
x = [total_latency, qi_total_latency, total_encoder_time, qi_total_encoder_time, total_decoder_time, qi_total_decoder_time]
y = ['Float32-Total', 'IntegerOps-Total', 'Float32-Encoder', 'IntegerOps-Encoder', 'Float32-Decoder', 'IntegerOps-Decoder',]
title = "Latency Results"
x_label = "Model Type"
y_label = "Latency (sec)"

plot_statistic(x, y, title, x_label, y_label)